In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using RigidBodyTreeInspector
using DrakeVisualizer
using Plots

using BilevelTrajOpt

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [30]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
env = parse_contacts(mechanism, urdf, .1);
x0 = MechanismState(mechanism)
Δt = 0.01;

In [3]:
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1));
geometries = visual_elements(mechanism, URDFVisuals(urdf))
vis = Visualizer(mechanism, geometries);

In [31]:
q0 = [1., 0., 0., 0., 0., 0., .05]
v0 = [0., 0., 0., 1., 0. , 0.]
N = 30

set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)

traj = BilevelTrajOpt.simulate(x0,env,Δt,N)

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      247
Number of nonzeros in inequality constraint Jacobian.:      342
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       19
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       13
Total number of inequality constraints...............:       18
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       18

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

19×31 Array{Float64,2}:
 1.0    1.0           1.0          …   1.0           1.0        
 0.0   -6.25562e-41  -1.25283e-40     -2.0409e-39   -2.11236e-39
 0.0   -1.05879e-26  -3.17637e-26     -1.6623e-24   -1.72583e-24
 0.0   -2.36545e-42  -4.70269e-42      7.72001e-37   9.27189e-37
 0.0    0.01          0.02             0.252326      0.259383   
 0.0    9.16188e-25   6.07485e-25  …   4.61389e-18   4.9743e-18 
 0.05   0.049019      0.047057         4.49362e-7    2.08126e-7 
 0.0   -1.25112e-38  -1.25453e-38     -1.42915e-38  -1.42915e-38
 0.0   -2.11758e-24  -4.23516e-24     -1.27055e-23  -1.27055e-23
 0.0   -4.73091e-40  -4.67448e-40      3.09806e-35   3.10375e-35
 1.0    1.0           1.0          …   0.715506      0.705702   
 0.0    9.16188e-23  -3.08703e-23      3.48404e-17   3.60412e-17
 0.0   -0.0980989    -0.196198         3.09241e-5   -2.41235e-5 
 0.0   -3.41931e-7   -4.64086e-7      -7.26549e-8   -5.79363e-8 
 0.0   -1.62042e-7   -2.11404e-7       1.2888e-7     1.00518e-7 
 

In [32]:
qs = [traj[1:num_positions(x0),i] for i in 1:N]
ts = cumsum([Δt for i in 1:N]);
RigidBodyTreeInspector.animate(vis, mechanism, ts, qs; realtimerate = .5);